# Multi-reference ROUGE 평가 시스템 테스트

이 노트북은 대회 평가 방식인 3개의 정답 요약문에 대한 ROUGE 점수 계산을 테스트합니다.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# 상위 디렉토리의 code를 import 경로에 추가
sys.path.append(str(Path.cwd().parent / 'code'))

from utils.metrics import (
    RougeCalculator, 
    MultiReferenceROUGE,
    calculate_rouge_scores,
    compute_metrics_for_trainer,
    evaluate_competition_format
)

## 1. 테스트 데이터 준비

In [ ]:
# 테스트용 데이터 생성
test_predictions = [
    "날씨가 좋아 산책하기 좋은 날이라고 이야기했습니다.",
    "프로젝트가 순조롭게 진행되고 있으며 다음 주까지 완료 예정입니다.",
    "주말에 액션 영화를 보러 가기로 약속했습니다."
]

# 각 예측에 대한 3개의 참조 요약문
test_references = [
    # 첫 번째 예측에 대한 3개 참조
    [
        "날씨가 좋아서 산책하기 좋다고 말했습니다.",
        "좋은 날씨에 산책하면 좋겠다는 대화를 나눴습니다.",
        "날씨가 좋아 산책하기 딱 좋은 날이라고 이야기했습니다."
    ],
    # 두 번째 예측에 대한 3개 참조
    [
        "프로젝트가 잘 진행되고 있어 다음 주 완료 가능합니다.",
        "프로젝트 진행이 순조로워 다음 주까지 끝낼 수 있습니다.",
        "프로젝트가 순조롭게 진행 중이며 다음 주 완료 예정입니다."
    ],
    # 세 번째 예측에 대한 3개 참조
    [
        "주말에 영화를 보러 가기로 했습니다.",
        "액션 영화를 주말에 함께 보기로 약속했습니다.",
        "주말에 액션 영화를 보러 가기로 했습니다."
    ]
]

print(f"예측 수: {len(test_predictions)}")
print(f"각 예측당 참조 수: {len(test_references[0])}")

## 2. 단일 참조 vs 다중 참조 비교

In [ ]:
# RougeCalculator 생성
calculator = RougeCalculator(use_korean_tokenizer=True)

# 첫 번째 예측에 대해 단일 참조와 다중 참조 비교
prediction = test_predictions[0]
references = test_references[0]

print("예측:", prediction)
print("\n참조 요약문들:")
for i, ref in enumerate(references):
    print(f"  {i+1}. {ref}")

print("\n=== 단일 참조 ROUGE 점수 ===")
for i, ref in enumerate(references):
    score = calculator.calculate_single_reference(prediction, ref)
    print(f"\n참조 {i+1}과의 점수:")
    print(f"  ROUGE-1 F1: {score.rouge1.f1:.4f}")
    print(f"  ROUGE-2 F1: {score.rouge2.f1:.4f}")
    print(f"  ROUGE-L F1: {score.rougeL.f1:.4f}")
    print(f"  Combined: {score.rouge_combined_f1:.4f}")

print("\n=== 다중 참조 ROUGE 점수 (최고 점수 선택) ===")
multi_score = calculator.calculate_multi_reference(prediction, references)
print(f"ROUGE-1 F1: {multi_score.rouge1.f1:.4f}")
print(f"ROUGE-2 F1: {multi_score.rouge2.f1:.4f}")
print(f"ROUGE-L F1: {multi_score.rougeL.f1:.4f}")
print(f"Combined: {multi_score.rouge_combined_f1:.4f}")

## 3. 전체 데이터에 대한 다중 참조 평가

In [ ]:
# 모든 예측에 대한 다중 참조 평가
all_scores = []

for pred, refs in zip(test_predictions, test_references):
    score = calculator.calculate_multi_reference(pred, refs)
    all_scores.append(score)

# 평균 점수 계산
avg_rouge1_f1 = np.mean([s.rouge1.f1 for s in all_scores])
avg_rouge2_f1 = np.mean([s.rouge2.f1 for s in all_scores])
avg_rougeL_f1 = np.mean([s.rougeL.f1 for s in all_scores])
avg_combined_f1 = avg_rouge1_f1 + avg_rouge2_f1 + avg_rougeL_f1

print("=== 전체 데이터 평균 점수 ===")
print(f"ROUGE-1 F1: {avg_rouge1_f1:.4f}")
print(f"ROUGE-2 F1: {avg_rouge2_f1:.4f}")
print(f"ROUGE-L F1: {avg_rougeL_f1:.4f}")
print(f"Combined F1 (대회 점수): {avg_combined_f1:.4f}")

# 개별 점수 확인
print("\n=== 개별 샘플 점수 ===")
for i, score in enumerate(all_scores):
    print(f"\n샘플 {i+1}:")
    print(f"  Combined F1: {score.rouge_combined_f1:.4f}")
    print(f"  (R1: {score.rouge1.f1:.4f}, R2: {score.rouge2.f1:.4f}, RL: {score.rougeL.f1:.4f})")

## 4. HuggingFace Trainer 호환 테스트

In [ ]:
# 가상의 토크나이저 (테스트용)
class DummyTokenizer:
    def __init__(self):
        self.pad_token_id = 0
    
    def batch_decode(self, token_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True):
        # 테스트를 위해 미리 정의된 텍스트 반환
        return test_predictions[:len(token_ids)]

# 토크나이저와 compute_metrics 함수 생성
dummy_tokenizer = DummyTokenizer()

# 단일 참조용 compute_metrics
compute_metrics_single = compute_metrics_for_trainer(
    dummy_tokenizer, 
    use_korean_tokenizer=True,
    multi_reference=False
)

# 다중 참조용 compute_metrics
compute_metrics_multi = compute_metrics_for_trainer(
    dummy_tokenizer,
    use_korean_tokenizer=True,
    multi_reference=True
)

print("compute_metrics 함수 생성 완료")
print(f"- 단일 참조: {type(compute_metrics_single)}")
print(f"- 다중 참조: {type(compute_metrics_multi)}")

## 5. 대회 형식 평가 테스트

In [ ]:
# 대회 형식의 테스트 데이터 생성
# 예측 파일 (fname, summary)
pred_data = pd.DataFrame({
    'fname': ['dialogue_001.txt', 'dialogue_002.txt', 'dialogue_003.txt'],
    'summary': test_predictions
})

# 정답 파일 (fname, summary1, summary2, summary3)
truth_data = pd.DataFrame({
    'fname': ['dialogue_001.txt', 'dialogue_002.txt', 'dialogue_003.txt'],
    'summary1': [refs[0] for refs in test_references],
    'summary2': [refs[1] for refs in test_references],
    'summary3': [refs[2] for refs in test_references]
})

# 임시 파일로 저장
pred_file = 'test_predictions.csv'
truth_file = 'test_ground_truth.csv'

pred_data.to_csv(pred_file, index=False)
truth_data.to_csv(truth_file, index=False)

print("예측 파일:")
print(pred_data)
print("\n정답 파일:")
print(truth_data)

In [ ]:
# 대회 형식 평가 실행
competition_results = evaluate_competition_format(
    pred_file,
    truth_file,
    use_korean_tokenizer=True
)

# 이전 결과와 비교
print("\n=== 결과 비교 ===")
print(f"수동 계산 Combined F1: {avg_combined_f1:.4f}")
print(f"함수 계산 Combined F1: {competition_results['rouge_combined_f1']:.4f}")
print(f"차이: {abs(avg_combined_f1 - competition_results['rouge_combined_f1']):.6f}")

In [ ]:
# 테스트 파일 정리
import os
os.remove(pred_file)
os.remove(truth_file)
print("테스트 파일 삭제 완료")

## 6. 극단적인 케이스 테스트

In [ ]:
# 극단적인 케이스들
edge_cases = [
    # 케이스 1: 완벽한 일치
    {
        'prediction': "프로젝트가 성공적으로 완료되었습니다.",
        'references': [
            "프로젝트가 성공적으로 완료되었습니다.",
            "프로젝트가 성공적으로 완료되었습니다.",
            "프로젝트가 성공적으로 완료되었습니다."
        ]
    },
    # 케이스 2: 완전히 다른 내용
    {
        'prediction': "날씨가 좋습니다.",
        'references': [
            "프로젝트를 진행합니다.",
            "회의를 시작합니다.",
            "보고서를 작성합니다."
        ]
    },
    # 케이스 3: 참조 중 하나만 유사
    {
        'prediction': "회의에서 예산을 논의했습니다.",
        'references': [
            "회의에서 예산을 논의했습니다.",
            "점심 메뉴를 정했습니다.",
            "주말 계획을 세웠습니다."
        ]
    },
    # 케이스 4: 빈 문자열
    {
        'prediction': "",
        'references': ["요약1", "요약2", "요약3"]
    }
]

print("=== 극단적인 케이스 테스트 ===")
for i, case in enumerate(edge_cases):
    print(f"\n케이스 {i+1}:")
    print(f"예측: '{case['prediction']}'")
    print(f"참조: {case['references']}")
    
    score = calculator.calculate_multi_reference(
        case['prediction'], 
        case['references']
    )
    
    print(f"결과: Combined F1 = {score.rouge_combined_f1:.4f}")
    print(f"      (R1: {score.rouge1.f1:.4f}, R2: {score.rouge2.f1:.4f}, RL: {score.rougeL.f1:.4f})")

## 7. 성능 측정

In [ ]:
import time

# 대량 데이터 생성
n_samples = 100
large_predictions = test_predictions * (n_samples // 3)
large_references = test_references * (n_samples // 3)

print(f"테스트 샘플 수: {len(large_predictions)}")

# 단일 참조 성능
start_time = time.time()
for pred, refs in zip(large_predictions, large_references):
    calculator.calculate_single_reference(pred, refs[0])
single_time = time.time() - start_time

# 다중 참조 성능
start_time = time.time()
for pred, refs in zip(large_predictions, large_references):
    calculator.calculate_multi_reference(pred, refs)
multi_time = time.time() - start_time

print(f"\n=== 성능 측정 결과 ===")
print(f"단일 참조 처리 시간: {single_time:.2f}초 ({single_time/len(large_predictions)*1000:.2f}ms/샘플)")
print(f"다중 참조 처리 시간: {multi_time:.2f}초 ({multi_time/len(large_predictions)*1000:.2f}ms/샘플)")
print(f"속도 차이: {multi_time/single_time:.1f}배")

## 결론

Multi-reference ROUGE 평가 시스템이 성공적으로 구현되었습니다:

1. **정확한 다중 참조 처리**: 3개의 참조 요약문 중 각 메트릭별로 최고 점수를 선택
2. **대회 평가 방식 준수**: rouge_combined_f1 = rouge1_f1 + rouge2_f1 + rougeL_f1
3. **HuggingFace Trainer 호환**: compute_metrics_for_trainer() 함수로 쉽게 통합
4. **대회 형식 평가 지원**: evaluate_competition_format() 함수로 제출 파일 직접 평가
5. **안정적인 처리**: 빈 문자열, 완벽한 일치 등 극단적인 케이스도 정상 처리

이제 대회의 평가 방식과 100% 일치하는 평가를 수행할 수 있습니다!